In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

# BIWI head pose dataset 回归任务
- 数据集的每个图片中有一个点，这个点代表脸部的中间位置
- 我们试图创建模型来找到这个中心点

In [2]:
from fastai.vision import *